## Prevendo os preços do Petróleo Brent do IPEA

In [3]:
import pandas as pd
from utils import atualizando_dados_ipea

In [4]:
path = atualizando_dados_ipea()
dados = pd.read_csv(path)

In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11088 entries, 0 to 11087
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Data                                  11088 non-null  object 
 1   Preço - petróleo bruto - Brent (FOB)  11088 non-null  float64
dtypes: float64(1), object(1)
memory usage: 173.4+ KB


In [6]:
dados.head()

,Data,Preço - petróleo bruto - Brent (FOB)
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
